# Introduction to Multi-Objective Decision Making


## Overview

Multi-Objective Decision Making (MODM) is a branch of operations research that deals with decision problems involving multiple, often conflicting objectives. Unlike single-objective optimization where we seek to maximize or minimize one criterion, MODM requires balancing trade-offs between competing goals.

### Learning Objectives

By the end of this notebook, you will understand:
- The fundamentals of multi-objective decision making
- Linear Programming review and its extension to multiple objectives
- Pareto optimality and efficient frontiers
- Goal Programming methodology
- Practical applications and solution methods

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

# Set visualization style
plt.style.use("seaborn-v0_8-darkgrid")
np.set_printoptions(precision=4, suppress=True)

## 1. Introduction to Multi-Objective Decision Making

### What is MODM?

In real-world decision making, we rarely optimize just one objective. Consider:
- **Manufacturing**: Minimize cost AND maximize quality
- **Investment**: Maximize return AND minimize risk
- **Engineering Design**: Maximize performance AND minimize weight AND minimize cost

### Key Characteristics

1. **Multiple Objectives**: Two or more objectives to optimize simultaneously
2. **Conflicting Goals**: Improving one objective may worsen another
3. **No Single Optimal Solution**: Instead, we have a set of "efficient" or "Pareto optimal" solutions
4. **Trade-offs**: Decision makers must balance competing objectives

### Mathematical Formulation

A general multi-objective optimization problem can be expressed as:

$$
\begin{align}
\text{Optimize} \quad & \mathbf{f}(\mathbf{x}) = [f_1(\mathbf{x}), f_2(\mathbf{x}), \ldots, f_k(\mathbf{x})]^T \\
\text{subject to} \quad & g_i(\mathbf{x}) \leq 0, \quad i = 1, 2, \ldots, m \\
& h_j(\mathbf{x}) = 0, \quad j = 1, 2, \ldots, p \\
& \mathbf{x} \in X
\end{align}
$$

Where:
- $\mathbf{x}$ is the decision variable vector
- $f_i(\mathbf{x})$ are the objective functions
- $g_i(\mathbf{x})$ are inequality constraints
- $h_j(\mathbf{x})$ are equality constraints
- $X$ is the feasible region

## 2. Review of Linear Programming

Before diving into multi-objective optimization, let's review single-objective Linear Programming (LP).

### Standard LP Form

$$
\begin{align}
\text{Maximize} \quad & \mathbf{c}^T \mathbf{x} \\
\text{subject to} \quad & A\mathbf{x} \leq \mathbf{b} \\
& \mathbf{x} \geq \mathbf{0}
\end{align}
$$

### Example: Production Planning

A factory produces two products, P1 and P2:
- P1 gives profit of \$40 per unit
- P2 gives profit of \$30 per unit

**Constraints:**
- Machine time: $2x_1 + x_2 \leq 100$ hours
- Labor time: $x_1 + x_2 \leq 80$ hours
- Material: $x_1 \leq 40$ units
- Non-negativity: $x_1, x_2 \geq 0$

In [ ]:
# Solve the single-objective LP problem
# Using scipy.optimize.linprog (minimization form)

# Objective: Maximize 40*x1 + 30*x2 => Minimize -40*x1 - 30*x2
c = [-40, -30]  # Coefficients for minimization

# Inequality constraints: A_ub @ x <= b_ub
A_ub = [
    [2, 1],  # Machine time
    [1, 1],  # Labor time
    [1, 0],  # Material
]
b_ub = [100, 80, 40]

# Bounds for variables (x >= 0)
x_bounds = [(0, None), (0, None)]

# Solve
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=x_bounds, method="highs")

print("=" * 50)
print("Single-Objective LP Solution")
print("=" * 50)
print(f"Optimal Solution: x1 = {result.x[0]:.2f}, x2 = {result.x[1]:.2f}")
print(f"Maximum Profit: ${-result.fun:.2f}")
print(f"Status: {result.message}")

In [ ]:
# Visualize the LP problem
fig, ax = plt.subplots(figsize=(10, 8))

# Define the feasible region
x1 = np.linspace(0, 50, 300)

# Constraint lines
y1 = 100 - 2 * x1  # Machine time
y2 = 80 - x1  # Labor time
y3 = np.full_like(x1, 100)  # Upper bound for plotting

# Plot constraints
ax.plot(x1, y1, "r-", label="Machine: 2x₁ + x₂ ≤ 100", linewidth=2)
ax.plot(x1, y2, "b-", label="Labor: x₁ + x₂ ≤ 80", linewidth=2)
ax.axvline(
    x=40, color="g", linestyle="-", label="Material: x₁ ≤ 40", linewidth=2
)

# Fill feasible region
y_feasible = np.minimum(np.minimum(y1, y2), y3)
y_feasible = np.maximum(y_feasible, 0)
x1_feasible = x1[x1 <= 40]
y_feasible = y_feasible[: len(x1_feasible)]
ax.fill_between(
    x1_feasible,
    0,
    y_feasible,
    alpha=0.3,
    color="yellow",
    label="Feasible Region",
)

# Plot optimal solution
ax.plot(
    result.x[0],
    result.x[1],
    "ro",
    markersize=12,
    label=f"Optimal: ({result.x[0]:.1f}, {result.x[1]:.1f})",
    zorder=5,
)

# Plot objective function contours
x1_grid, x2_grid = np.meshgrid(
    np.linspace(0, 50, 100), np.linspace(0, 100, 100)
)
profit = 40 * x1_grid + 30 * x2_grid
contours = ax.contour(
    x1_grid, x2_grid, profit, levels=10, alpha=0.4, cmap="viridis"
)
ax.clabel(contours, inline=True, fontsize=8)

ax.set_xlabel("x₁ (Product 1)", fontsize=12)
ax.set_ylabel("x₂ (Product 2)", fontsize=12)
ax.set_title(
    "Single-Objective Linear Programming Problem",
    fontsize=14,
    fontweight="bold",
)
ax.legend(loc="upper right")
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 50)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()

### Key Observations from Single-Objective LP

1. **Unique Optimal Solution**: There is typically one optimal point (or a line segment in degenerate cases)
2. **Corner Point Theorem**: The optimal solution occurs at a vertex of the feasible region
3. **Sensitivity**: Small changes in constraints or objectives can shift the optimal solution

### Transition to Multi-Objective

In the single-objective case, we had one clear winner. But what if we want to:
- Maximize profit AND minimize resource usage?
- Maximize profit AND maximize product quality?
- Minimize cost AND minimize production time?

This is where **multi-objective optimization** becomes essential!

## 3. Pareto Optimization

### Pareto Dominance

A solution $\mathbf{x}^1$ **dominates** solution $\mathbf{x}^2$ if:
- $\mathbf{x}^1$ is at least as good as $\mathbf{x}^2$ in all objectives
- $\mathbf{x}^1$ is strictly better than $\mathbf{x}^2$ in at least one objective

Mathematically (for minimization):
$$
\mathbf{x}^1 \prec \mathbf{x}^2 \iff f_i(\mathbf{x}^1) \leq f_i(\mathbf{x}^2) \; \forall i \text{ and } f_j(\mathbf{x}^1) < f_j(\mathbf{x}^2) \text{ for at least one } j
$$

### Pareto Optimal (Efficient) Solution

A solution is **Pareto optimal** if it is not dominated by any other feasible solution. The set of all Pareto optimal solutions forms the **Pareto frontier** (or **efficient frontier**).

### Example: Bi-objective Problem

Consider a company that wants to:
- **Objective 1**: Maximize profit: $f_1 = 40x_1 + 30x_2$
- **Objective 2**: Minimize resource usage: $f_2 = 2x_1 + 3x_2$

Subject to the same constraints as before.

In [ ]:
# Generate candidate solutions in the feasible region
def is_feasible(x1, x2):
    """Check if a point is feasible"""
    return (
        (2 * x1 + x2 <= 100)
        and (x1 + x2 <= 80)
        and (x1 <= 40)
        and (x1 >= 0)
        and (x2 >= 0)
    )


# Create a grid of points
x1_range = np.linspace(0, 45, 50)
x2_range = np.linspace(0, 85, 50)
X1, X2 = np.meshgrid(x1_range, x2_range)

# Calculate objectives for all points
solutions = []
for i in range(len(x1_range)):
    for j in range(len(x2_range)):
        x1, x2 = X1[j, i], X2[j, i]
        if is_feasible(x1, x2):
            profit = 40 * x1 + 30 * x2
            resource = 2 * x1 + 3 * x2
            solutions.append([x1, x2, profit, resource])

solutions = np.array(solutions)
print(f"Generated {len(solutions)} feasible solutions")

In [ ]:
# Find Pareto optimal solutions
def is_dominated(solution, all_solutions):
    """Check if a solution is dominated by any other solution"""
    profit, resource = solution[2], solution[3]
    for other in all_solutions:
        other_profit, other_resource = other[2], other[3]
        # For maximization profit and minimization resource
        if other_profit >= profit and other_resource <= resource:
            if other_profit > profit or other_resource < resource:
                return True
    return False


# Find Pareto frontier
pareto_solutions = []
for sol in solutions:
    if not is_dominated(sol, solutions):
        pareto_solutions.append(sol)

pareto_solutions = np.array(pareto_solutions)
print(f"Found {len(pareto_solutions)} Pareto optimal solutions")

# Sort by profit for better visualization
pareto_solutions = pareto_solutions[pareto_solutions[:, 2].argsort()]

In [ ]:
# Visualize Pareto frontier in objective space
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Decision space
ax1.scatter(
    solutions[:, 0],
    solutions[:, 1],
    c="lightblue",
    s=30,
    alpha=0.5,
    label="Feasible Solutions",
)
ax1.scatter(
    pareto_solutions[:, 0],
    pareto_solutions[:, 1],
    c="red",
    s=50,
    marker="*",
    label="Pareto Optimal Solutions",
    zorder=5,
)
ax1.set_xlabel("x₁ (Product 1)", fontsize=12)
ax1.set_ylabel("x₂ (Product 2)", fontsize=12)
ax1.set_title("Decision Space", fontsize=14, fontweight="bold")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right plot: Objective space
ax2.scatter(
    solutions[:, 2],
    solutions[:, 3],
    c="lightblue",
    s=30,
    alpha=0.5,
    label="Feasible Solutions",
)
ax2.scatter(
    pareto_solutions[:, 2],
    pareto_solutions[:, 3],
    c="red",
    s=50,
    marker="*",
    label="Pareto Frontier",
    zorder=5,
)
ax2.plot(
    pareto_solutions[:, 2],
    pareto_solutions[:, 3],
    "r--",
    alpha=0.6,
    linewidth=2,
)
ax2.set_xlabel("f₁: Profit ($)", fontsize=12)
ax2.set_ylabel("f₂: Resource Usage", fontsize=12)
ax2.set_title(
    "Objective Space (Pareto Frontier)", fontsize=14, fontweight="bold"
)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Add arrow indicating preference direction
ax2.annotate(
    "",
    xy=(ax2.get_xlim()[1] * 0.95, ax2.get_ylim()[0] * 1.05),
    xytext=(ax2.get_xlim()[1] * 0.85, ax2.get_ylim()[0] * 1.05),
    arrowprops=dict(arrowstyle="->", color="green", lw=2),
)
ax2.text(
    ax2.get_xlim()[1] * 0.90,
    ax2.get_ylim()[0] * 1.12,
    "Maximize",
    color="green",
    fontsize=10,
    ha="center",
)

ax2.annotate(
    "",
    xy=(ax2.get_xlim()[0] * 1.05, ax2.get_ylim()[1] * 0.5),
    xytext=(ax2.get_xlim()[0] * 1.05, ax2.get_ylim()[1] * 0.6),
    arrowprops=dict(arrowstyle="->", color="green", lw=2),
)
ax2.text(
    ax2.get_xlim()[0] * 1.15,
    ax2.get_ylim()[1] * 0.55,
    "Minimize",
    color="green",
    fontsize=10,
    rotation=90,
    va="center",
)

plt.tight_layout()
plt.show()

In [ ]:
# Display some Pareto optimal solutions
pareto_df = pd.DataFrame(
    pareto_solutions, columns=["x₁", "x₂", "Profit ($)", "Resource Usage"]
)
pareto_df = pareto_df.sort_values("Profit ($)", ascending=False)

print("\n" + "=" * 60)
print("Sample Pareto Optimal Solutions")
print("=" * 60)
print(pareto_df.head(10).to_string(index=False))
print("\n" + "=" * 60)
print("Trade-off Analysis:")
print("=" * 60)
print(
    f"Highest Profit Solution: ${pareto_df['Profit ($)'].max():.2f} with Resource Usage: {pareto_df.loc[pareto_df['Profit ($)'].idxmax(), 'Resource Usage']:.2f}"
)
print(
    f"Lowest Resource Solution: Resource = {pareto_df['Resource Usage'].min():.2f} with Profit: ${pareto_df.loc[pareto_df['Resource Usage'].idxmin(), 'Profit ($)']:.2f}"
)

### Key Insights on Pareto Optimality

1. **No Single Best Solution**: Unlike single-objective optimization, there's no universally "best" solution
2. **Trade-offs**: Moving along the Pareto frontier involves trading one objective for another
3. **Decision Maker's Role**: The final choice depends on the decision maker's preferences
4. **Dominated Solutions**: Any solution not on the Pareto frontier is suboptimal

### Common Methods to Find Pareto Solutions

1. **Weighted Sum Method**: $\min \sum_{i=1}^{k} w_i f_i(\mathbf{x})$
2. **ε-Constraint Method**: Optimize one objective while constraining others
3. **Evolutionary Algorithms**: NSGA-II, SPEA2, MOEA/D
4. **Goal Programming**: Minimize deviations from target goals (discussed next)

## 4. Goal Programming

### What is Goal Programming?

Goal Programming (GP) is a multi-objective optimization technique where decision makers specify **target values (goals)** for each objective, and the optimization seeks to minimize deviations from these goals.

### Key Concepts

1. **Goals**: Target values for objectives ($g_i$)
2. **Deviational Variables**: 
   - $d_i^+$: Positive deviation (over-achievement)
   - $d_i^-$: Negative deviation (under-achievement)
3. **Priority Levels**: Goals can be ranked by importance ($P_1, P_2, \ldots$)

### Mathematical Formulation

$$
\begin{align}
\text{Minimize} \quad & Z = \sum_{i=1}^{k} w_i^+ d_i^+ + w_i^- d_i^- \\
\text{subject to} \quad & f_i(\mathbf{x}) + d_i^- - d_i^+ = g_i, \quad i = 1, 2, \ldots, k \\
& \text{original constraints} \\
& d_i^+, d_i^- \geq 0, \quad i = 1, 2, \ldots, k
\end{align}
$$

Where $w_i^+$ and $w_i^-$ are penalty weights for over- and under-achievement.

### Example: Production Planning with Goals

Using our previous example, suppose management sets the following goals:
- **Goal 1**: Achieve profit of at least \$2500 (priority: high)
- **Goal 2**: Keep resource usage below 150 units (priority: medium)
- **Goal 3**: Produce at least 30 units of Product 1 (priority: low)

In [ ]:
# Goal Programming formulation
# Decision variables: [x1, x2, d1-, d1+, d2-, d2+, d3-, d3+]
# where di- is under-achievement and di+ is over-achievement for goal i

from scipy.optimize import linprog

# Objective: Minimize weighted deviations
# Priority weights: P1=100, P2=10, P3=1
# We penalize: under-achievement of profit (d1-), over-achievement of resources (d2+), under-achievement of x1 (d3-)
c_gp = [
    0,
    0,  # x1, x2 (no direct cost)
    100,
    0,  # d1-, d1+ (penalize under-achievement of profit heavily)
    0,
    10,  # d2-, d2+ (penalize over-achievement of resource usage)
    1,
    0,
]  # d3-, d3+ (penalize under-achievement of x1 production)

# Constraints
# 1. Original constraints (machine, labor, material)
# 2. Goal constraints:
#    - 40*x1 + 30*x2 + d1- - d1+ = 2500  (profit goal)
#    - 2*x1 + 3*x2 + d2- - d2+ = 150     (resource goal)
#    - x1 + d3- - d3+ = 30               (production goal)

# Inequality constraints (A_ub @ x <= b_ub)
A_ub_gp = [
    [2, 1, 0, 0, 0, 0, 0, 0],  # Machine time: 2*x1 + x2 <= 100
    [1, 1, 0, 0, 0, 0, 0, 0],  # Labor time: x1 + x2 <= 80
    [1, 0, 0, 0, 0, 0, 0, 0],  # Material: x1 <= 40
]
b_ub_gp = [100, 80, 40]

# Equality constraints (A_eq @ x = b_eq)
A_eq_gp = [
    [
        40,
        30,
        1,
        -1,
        0,
        0,
        0,
        0,
    ],  # Profit goal: 40*x1 + 30*x2 + d1- - d1+ = 2500
    [2, 3, 0, 0, 1, -1, 0, 0],  # Resource goal: 2*x1 + 3*x2 + d2- - d2+ = 150
    [1, 0, 0, 0, 0, 0, 1, -1],  # Production goal: x1 + d3- - d3+ = 30
]
b_eq_gp = [2500, 150, 30]

# Bounds (all variables >= 0)
bounds_gp = [(0, None)] * 8

# Solve
result_gp = linprog(
    c_gp,
    A_ub=A_ub_gp,
    b_ub=b_ub_gp,
    A_eq=A_eq_gp,
    b_eq=b_eq_gp,
    bounds=bounds_gp,
    method="highs",
)

# Extract results
x1_gp, x2_gp = result_gp.x[0], result_gp.x[1]
d1_minus, d1_plus = result_gp.x[2], result_gp.x[3]
d2_minus, d2_plus = result_gp.x[4], result_gp.x[5]
d3_minus, d3_plus = result_gp.x[6], result_gp.x[7]

actual_profit = 40 * x1_gp + 30 * x2_gp
actual_resource = 2 * x1_gp + 3 * x2_gp

print("\n" + "=" * 60)
print("Goal Programming Solution")
print("=" * 60)
print(f"\nDecision Variables:")
print(f"  x₁ (Product 1) = {x1_gp:.2f} units")
print(f"  x₂ (Product 2) = {x2_gp:.2f} units")
print(f"\nGoal Achievement:")
print(f"  Goal 1 - Profit:")
print(f"    Target: $2500.00, Actual: ${actual_profit:.2f}")
print(f"    Under-achievement (d₁⁻): ${d1_minus:.2f}")
print(f"    Over-achievement (d₁⁺): ${d1_plus:.2f}")
print(f"\n  Goal 2 - Resource Usage:")
print(f"    Target: 150.00, Actual: {actual_resource:.2f}")
print(f"    Under-achievement (d₂⁻): {d2_minus:.2f}")
print(f"    Over-achievement (d₂⁺): {d2_plus:.2f}")
print(f"\n  Goal 3 - Product 1 Production:")
print(f"    Target: 30.00, Actual: {x1_gp:.2f}")
print(f"    Under-achievement (d₃⁻): {d3_minus:.2f}")
print(f"    Over-achievement (d₃⁺): {d3_plus:.2f}")
print(f"\nTotal Weighted Deviation: {result_gp.fun:.2f}")
print("=" * 60)

In [ ]:
# Visualize Goal Programming solution vs Pareto frontier
fig, ax = plt.subplots(figsize=(12, 8))

# Plot all feasible solutions
ax.scatter(
    solutions[:, 2],
    solutions[:, 3],
    c="lightblue",
    s=30,
    alpha=0.5,
    label="Feasible Solutions",
)

# Plot Pareto frontier
ax.plot(
    pareto_solutions[:, 2],
    pareto_solutions[:, 3],
    "r--",
    alpha=0.6,
    linewidth=2,
    label="Pareto Frontier",
)
ax.scatter(
    pareto_solutions[:, 2],
    pareto_solutions[:, 3],
    c="red",
    s=50,
    marker="*",
    zorder=5,
)

# Plot Goal Programming solution
ax.scatter(
    actual_profit,
    actual_resource,
    c="green",
    s=200,
    marker="D",
    edgecolors="black",
    linewidths=2,
    label="Goal Programming Solution",
    zorder=10,
)

# Plot goal targets
ax.axvline(
    x=2500,
    color="orange",
    linestyle=":",
    linewidth=2,
    label="Profit Goal ($2500)",
    alpha=0.7,
)
ax.axhline(
    y=150,
    color="purple",
    linestyle=":",
    linewidth=2,
    label="Resource Goal (150)",
    alpha=0.7,
)

# Ideal point
ax.scatter(
    2500,
    150,
    c="gold",
    s=150,
    marker="*",
    edgecolors="black",
    linewidths=1.5,
    label="Goal Target Point",
    zorder=8,
)

ax.set_xlabel("f₁: Profit ($)", fontsize=12)
ax.set_ylabel("f₂: Resource Usage", fontsize=12)
ax.set_title(
    "Goal Programming Solution in Objective Space",
    fontsize=14,
    fontweight="bold",
)
ax.legend(loc="upper right")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Variants of Goal Programming

#### 1. **Weighted Goal Programming**
All goals are considered simultaneously with different weights:
$$Z = \sum_{i=1}^{k} (w_i^+ d_i^+ + w_i^- d_i^-)$$

#### 2. **Lexicographic (Preemptive) Goal Programming**
Goals are ordered by priority levels. Higher priority goals are satisfied before lower ones:
$$Z = P_1(\ldots) + P_2(\ldots) + P_3(\ldots)$$
where $P_1 \gg P_2 \gg P_3$

#### 3. **Minmax Goal Programming**
Minimize the maximum deviation:
$$Z = \min \max_i \{w_i^+ d_i^+ + w_i^- d_i^-\}$$

### Advantages of Goal Programming

1. ✅ Intuitive for decision makers (set targets)
2. ✅ Handles incommensurable objectives (different units)
3. ✅ Flexible priority structures
4. ✅ Always produces a feasible solution
5. ✅ Can incorporate soft constraints

### Limitations

1. ❌ Requires goal values (may be arbitrary)
2. ❌ Solution may not be Pareto optimal
3. ❌ Sensitive to weight selection
4. ❌ May not explore full trade-off space

## 5. Comparison of Methods

Let's compare different approaches to our multi-objective problem:

In [ ]:
# Compare different solution methods
comparison_data = {
    "Method": [
        "Single-Objective (Max Profit)",
        "Goal Programming",
        "Max Profit on Pareto Frontier",
        "Min Resource on Pareto Frontier",
        "Balanced Pareto Solution",
    ],
    "x₁": [
        result.x[0],
        x1_gp,
        pareto_solutions[pareto_solutions[:, 2].argmax(), 0],
        pareto_solutions[pareto_solutions[:, 3].argmin(), 0],
        pareto_solutions[len(pareto_solutions) // 2, 0],
    ],
    "x₂": [
        result.x[1],
        x2_gp,
        pareto_solutions[pareto_solutions[:, 2].argmax(), 1],
        pareto_solutions[pareto_solutions[:, 3].argmin(), 1],
        pareto_solutions[len(pareto_solutions) // 2, 1],
    ],
}

# Calculate objectives for each method
comparison_data["Profit ($)"] = [
    40 * x1 + 30 * x2
    for x1, x2 in zip(comparison_data["x₁"], comparison_data["x₂"])
]
comparison_data["Resource Usage"] = [
    2 * x1 + 3 * x2
    for x1, x2 in zip(comparison_data["x₁"], comparison_data["x₂"])
]

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "=" * 80)
print("Comparison of Multi-Objective Solution Methods")
print("=" * 80)
print(comparison_df.to_string(index=False))
print("=" * 80)

# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 8))

# Plot Pareto frontier
ax.plot(
    pareto_solutions[:, 2],
    pareto_solutions[:, 3],
    "r--",
    alpha=0.6,
    linewidth=2,
    label="Pareto Frontier",
)

# Plot different solutions
colors = ["blue", "green", "red", "purple", "orange"]
markers = ["o", "D", "*", "s", "^"]

for idx, (method, color, marker) in enumerate(
    zip(comparison_data["Method"], colors, markers)
):
    ax.scatter(
        comparison_data["Profit ($)"][idx],
        comparison_data["Resource Usage"][idx],
        c=color,
        s=150,
        marker=marker,
        edgecolors="black",
        linewidths=1.5,
        label=method,
        zorder=5,
    )

ax.set_xlabel("Profit ($)", fontsize=12)
ax.set_ylabel("Resource Usage", fontsize=12)
ax.set_title(
    "Comparison of Multi-Objective Optimization Methods",
    fontsize=14,
    fontweight="bold",
)
ax.legend(loc="upper right", fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Practical Applications

### Real-World Examples

#### 1. **Supply Chain Management**
- Minimize cost, minimize delivery time, maximize service level
- Trade-offs between inventory costs and stockout risks

#### 2. **Portfolio Optimization**
- Maximize expected return, minimize risk (variance)
- Balance between aggressive growth and capital preservation

#### 3. **Engineering Design**
- Minimize weight, maximize strength, minimize cost
- Aircraft design, structural optimization

#### 4. **Environmental Planning**
- Maximize economic benefit, minimize environmental impact
- Sustainable resource management

#### 5. **Healthcare Resource Allocation**
- Maximize patient satisfaction, minimize waiting time, minimize cost
- Hospital scheduling and resource planning

### Implementation Considerations

1. **Problem Definition**: Clearly identify all objectives and constraints
2. **Objective Normalization**: Scale objectives to comparable ranges
3. **Preference Elicitation**: Understand decision maker's priorities
4. **Solution Method Selection**: Choose appropriate technique based on problem characteristics
5. **Sensitivity Analysis**: Examine robustness of solutions
6. **Decision Support**: Present results in interpretable formats

## 7. Summary and Key Takeaways

### Main Concepts

1. **Multi-Objective Decision Making**: Optimizing multiple, often conflicting objectives simultaneously

2. **Pareto Optimality**: Solutions where improving one objective worsens another
   - No single "best" solution
   - Set of efficient solutions forms the Pareto frontier
   - Decision maker selects based on preferences

3. **Goal Programming**: Minimize deviations from predefined target goals
   - Intuitive for practitioners
   - Handles multiple priorities
   - May not guarantee Pareto optimality

### Method Selection Guide

| Method | Use When | Advantages | Disadvantages |
|--------|----------|------------|---------------|
| **Weighted Sum** | Objectives commensurable, convex problems | Simple, efficient | May miss non-convex Pareto points |
| **ε-Constraint** | Want to explore Pareto frontier | Finds non-convex points | Multiple LP solves needed |
| **Goal Programming** | Clear targets available | Intuitive, flexible | Arbitrary goals, may not be Pareto optimal |
| **Evolutionary Algorithms** | Complex, non-linear problems | Handles any problem type | Computationally expensive |

### Next Steps

To deepen your understanding:
1. Study advanced Pareto frontier generation techniques
2. Explore interactive decision-making methods
3. Learn about robust multi-objective optimization
4. Apply to real-world case studies in your domain

### Further Reading

- **Textbooks**:
  - Ehrgott, M. (2005). *Multicriteria Optimization*
  - Miettinen, K. (1999). *Nonlinear Multiobjective Optimization*
  - Jones, D., & Tamiz, M. (2010). *Practical Goal Programming*

- **Software Libraries**:
  - Python: `pymoo`, `platypus`, `DEAP`
  - MATLAB: Multi-Objective Optimization Toolbox
  - R: `mco`, `ecr`

## 8. Practice Exercises

### Exercise 1: Bi-objective Production Planning
A company produces two products with the following characteristics:
- Product A: Profit = $50/unit, Production time = 3 hours/unit
- Product B: Profit = $40/unit, Production time = 2 hours/unit
- Available time: 120 hours

**Objectives**: 
1. Maximize total profit
2. Minimize total production time

Find and visualize the Pareto frontier.

### Exercise 2: Goal Programming Application
Using the same problem from Exercise 1, apply goal programming with the following goals:
- Goal 1: Achieve profit of at least $1800 (Priority 1)
- Goal 2: Use no more than 100 hours (Priority 2)
- Goal 3: Produce at least 20 units of Product A (Priority 3)

Compare the solution with the Pareto frontier.

### Exercise 3: Three-Objective Problem
Extend the problem to three objectives:
1. Maximize profit
2. Minimize production time
3. Minimize material cost (Product A: $10/unit, Product B: $8/unit)

Visualize the 3D Pareto frontier and identify interesting trade-offs.

---

**End of Notebook**

*For questions or feedback, please contact your instructor.*